In [ ]:
import numpy as np
import volumentations
import nibabel as nib

def get_augmentation(patch_size):
    return volumentations.Compose([
        volumentations.Rotate((-15, 15), (0, 0), (0, 0), p=0.5),
        volumentations.RandomCropFromBorders(crop_value=0.1, p=0.5),
        volumentations.ElasticTransform((0, 0.25), interpolation=2, p=0.1),
        volumentations.Resize(patch_size, interpolation=1, resize_type=0, always_apply=True, p=1.0),
        volumentations.Flip(0, p=0.5),
        volumentations.Flip(1, p=0.5),
        volumentations.Flip(2, p=0.5),
        volumentations.RandomRotate90((1, 2), p=0.5),
        volumentations.GaussianNoise(var_limit=(0, 5), p=0.2),
        volumentations.RandomGamma(gamma_limit=(80, 120), p=0.2),
    ], p=1.0)

In [ ]:
def load_nii_file(file_path):
    """Load a .nii.gz file and return the image data"""
    nii_img = nib.load(file_path)
    return nii_img.get_fdata()

def normalize_volume(volume):
    """Normalize volume to [0, 1] range"""
    volume = volume.astype(np.float32)  # Keep as float32 for preprocessing
    # Remove outliers using percentile clipping
    p1, p99 = np.percentile(volume[volume > 0], [1, 99])
    volume = np.clip(volume, p1, p99)
    
    # Normalize to [0, 1]
    volume = (volume - volume.min()) / (volume.max() - volume.min() + 1e-8)
    return volume

def resize_volume(volume, target_shape):
    """Resize volume to target shape using simple interpolation"""
    from scipy.ndimage import zoom
    
    # Calculate zoom factors for each dimension
    zoom_factors = [target_shape[i] / volume.shape[i] for i in range(3)]
    
    # Resize the volume
    resized_volume = zoom(volume, zoom_factors, order=1)
    return resized_volume

def augment_volume(volume):
    """Simple augmentation: random rotation around axes"""
    # Get augmentation pipeline
    augment = get_augmentation(volume.shape)
    augmented_volume = augment(image=volume)["image"]
    # if random.random() > 0.5:
    #     # Random rotation around z-axis
    #     k = random.randint(0, 3)
    #     volume = np.rot90(volume, k=k, axes=(0, 1))
    
    # if random.random() > 0.5:
    #     # Random flip
    #     axis = random.randint(0, 2)
    #     volume = np.flip(volume, axis=axis)
    
    return augmented_volume

In [ ]:
file_path = r"C:\Users\acer\Desktop\Project_TMJOA\Data\training_dataset\test\0\50-30909 R_adjustedBG.nii.gz"
target_shape = (224, 224, 224)

volume = load_nii_file(file_path)
print("Original shape:", volume.shape)
print("Original dtype:", volume.dtype)
print("=========================================")

volume = normalize_volume(volume)
print("Normalized shape:", volume.shape)
print("Normalized dtype:", volume.dtype)
print("=========================================")

volume = resize_volume(volume, target_shape)
print("Resized shape:", volume.shape)
print("Resized dtype:", volume.dtype)
print("=========================================")

volume = augment_volume(volume)
print("Augmented shape:", volume.shape)
print("Augmented dtype:", volume.dtype)
print("=========================================")

volume = np.expand_dims(volume, axis=-1)
print("Final shape:", volume.shape)
print("Final dtype:", volume.dtype)
            